### RFM的概念：精准营销、精细化运营，目的是为了服务好存量客户；
### 给不同的用户提供不同的个性化服务；
### 用户分群方法：从RFM三个角度对用户进行划分；
### R:recency最近一次消费距今的时间；京东/淘宝  半个月之内来过  高/低
### F:frequency消费的频率；半年之内有25次及以上消费  高/低
### M:money消费金额：  半年之内 消费总金额在2500元以上   高/低

### 统计分析思路：（可以分成8组：）
### 只需要有用户消费流水的数据，就可以计算RFM
### 用户ID   消费金额    消费时间（历史几年的数据）
## 计算步骤：1、分组聚合，计算每个用户的RFM的聚合值
### R每一个用户最近一次消费时间距今的时间间隔（用户ID分组，对消费时间取最大max)
### F最近一段时间内每个用户消费的次数(用户ID分组，对数据计数count)
### M最近一段时间内每个用户消费的总金额(用户ID分组 金额求和sum)

## 2、给RFM进行分段，划分的依据由业务方提供；
## 3、将RFM三个维度的标签合并在一起；
## 4、对划分结果进行分析并可视化
    ### 每一组有多少人；是否合理等

In [2]:
import pandas as pd

In [3]:
# 读取数据
sheet_names = ['2015','2016','2017','2018']
sheet_datas = [pd.read_excel('./data/sales.xlsx', sheet_name=i) for i in sheet_names]

In [4]:
for each_name, each_data in zip(sheet_names, sheet_datas):
    print('===================================',each_name,'===========================================')
    print(each_data.describe())
    print(each_data.info())

=================================== 2015 ===========================================
               会员ID           订单号           订单金额
count  3.077400e+04  3.077400e+04   30774.000000
mean   2.918779e+10  4.020414e+09     960.991161
std    1.385333e+10  2.630510e+08    2068.107231
min    2.670000e+02  3.000305e+09       0.500000
25%    1.944122e+10  3.885510e+09      59.000000
50%    3.746545e+10  4.117491e+09     139.000000
75%    3.923593e+10  4.234882e+09     899.000000
max    3.954613e+10  4.282025e+09  111750.000000
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30774 entries, 0 to 30773
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   会员ID    30774 non-null  int64         
 1   订单号     30774 non-null  int64         
 2   提交日期    30774 non-null  datetime64[ns]
 3   订单金额    30774 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 961.8 KB
None
==========================

In [5]:
# 通过分析发现，数据中有缺失值，但不严重，有两年的数据分别有一条缺失值，这里可以做删除处理；
# 订单价格分布问题， 极大值和极小值差距太大，可以跟业务方沟通，询问业务方后这个价格合理，经过询问，可以删除价格小于1的订单
# 接下来做两件事：1、删除有缺失的数据；2、删除价格<1的订单

## 数据预处理

In [6]:
for index, each_data in enumerate(sheet_datas):
    sheet_datas[index] = each_data.dropna()
    sheet_datas[index] = each_data[each_data['订单金额']>1]
    sheet_datas[index]['max_year_date'] = each_data['提交日期'].max()

C:\Users\Admin\AppData\Local\Temp/ipykernel_11492/2531921916.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sheet_datas[index]['max_year_date'] = each_data['提交日期'].max()


In [7]:
# 查看第一个工作表数据，即2015年数据
sheet_datas[0]

,会员ID,订单号,提交日期,订单金额,max_year_date
0,15278002468,3000304681,2015-01-01,499.0,2015-12-31
1,39236378972,3000305791,2015-01-01,2588.0,2015-12-31
2,38722039578,3000641787,2015-01-01,498.0,2015-12-31
3,11049640063,3000798913,2015-01-01,1572.0,2015-12-31
4,35038752292,3000821546,2015-01-01,10.1,2015-12-31
...,...,...,...,...,...
30769,39368100847,4281994827,2015-12-31,828.0,2015-12-31
30770,409757,4282010457,2015-12-31,199.0,2015-12-31
30771,38380526114,4282017675,2015-12-31,208.0,2015-12-31
30772,28074988,4282019440,2015-12-31,89.0,2015-12-31


In [8]:
# 将不同年份的数据拼接在一起
data_merge = pd.concat(sheet_datas, axis=0)
data_merge

,会员ID,订单号,提交日期,订单金额,max_year_date
0,15278002468,3000304681,2015-01-01,499.0,2015-12-31
1,39236378972,3000305791,2015-01-01,2588.0,2015-12-31
2,38722039578,3000641787,2015-01-01,498.0,2015-12-31
3,11049640063,3000798913,2015-01-01,1572.0,2015-12-31
4,35038752292,3000821546,2015-01-01,10.1,2015-12-31
...,...,...,...,...,...
81344,39229485704,4354225182,2018-12-31,249.0,2018-12-31
81345,39229021075,4354225188,2018-12-31,89.0,2018-12-31
81346,39288976750,4354230034,2018-12-31,48.5,2018-12-31
81347,26772630,4354230163,2018-12-31,3196.0,2018-12-31


In [9]:
data_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 202827 entries, 0 to 81348
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   会员ID           202827 non-null  int64         
 1   订单号            202827 non-null  int64         
 2   提交日期           202827 non-null  datetime64[ns]
 3   订单金额           202827 non-null  float64       
 4   max_year_date  202827 non-null  datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(2)
memory usage: 9.3 MB


In [10]:
data_merge.columns

Index(['会员ID', '订单号', '提交日期', '订单金额', 'max_year_date'], dtype='object')

In [11]:
# 计算每一笔交易距今（统计时间节点）时间长；
data_merge['时间间隔'] = data_merge['max_year_date'] - data_merge['提交日期']
data_merge

,会员ID,订单号,提交日期,订单金额,max_year_date,时间间隔
0,15278002468,3000304681,2015-01-01,499.0,2015-12-31,364 days
1,39236378972,3000305791,2015-01-01,2588.0,2015-12-31,364 days
2,38722039578,3000641787,2015-01-01,498.0,2015-12-31,364 days
3,11049640063,3000798913,2015-01-01,1572.0,2015-12-31,364 days
4,35038752292,3000821546,2015-01-01,10.1,2015-12-31,364 days
...,...,...,...,...,...,...
81344,39229485704,4354225182,2018-12-31,249.0,2018-12-31,0 days
81345,39229021075,4354225188,2018-12-31,89.0,2018-12-31,0 days
81346,39288976750,4354230034,2018-12-31,48.5,2018-12-31,0 days
81347,26772630,4354230163,2018-12-31,3196.0,2018-12-31,0 days


In [12]:
data_merge['年份'] = data_merge['提交日期'].dt.year

In [13]:
data_merge['时间间隔'] = data_merge['时间间隔'].dt.days

In [14]:
data_merge.head()

,会员ID,订单号,提交日期,订单金额,max_year_date,时间间隔,年份
0,15278002468,3000304681,2015-01-01,499.0,2015-12-31,364,2015
1,39236378972,3000305791,2015-01-01,2588.0,2015-12-31,364,2015
2,38722039578,3000641787,2015-01-01,498.0,2015-12-31,364,2015
3,11049640063,3000798913,2015-01-01,1572.0,2015-12-31,364,2015
4,35038752292,3000821546,2015-01-01,10.1,2015-12-31,364,2015


## 4、分组聚合计算每个用户RFM的聚合值

In [15]:
# groupby默认会使用分组字段，作为聚合结果的行索引，如果as_index=False分组的字段在最终结果中只会作为普通列；
# 想实现跟as_index=False相同的效果，也可以不设置这个参数，对分组聚合之后的结果调用reset_index();
# agg聚合，可以对多个字段按照不同的聚合方式进行聚合，方法是接受一个字典{'字段名':'聚合方式'}；
rfm_result = data_merge.groupby(['年份', '会员ID'], as_index=False).agg({'时间间隔':'min', '订单号':'count', '订单金额':'sum'})
rfm_result

,年份,会员ID,时间间隔,订单号,订单金额
0,2015,267,197,2,105.0
1,2015,282,251,1,29.7
2,2015,283,340,1,5398.0
3,2015,343,300,1,118.0
4,2015,525,37,3,213.0
...,...,...,...,...,...
148586,2018,39538034299,272,1,49.0
148587,2018,39538034662,189,1,3558.0
148588,2018,39538035729,179,1,3699.0
148589,2018,39545237824,275,1,49.0


In [16]:
data_merge[data_merge['会员ID']==525]  # 验证以上数据

,会员ID,订单号,提交日期,订单金额,max_year_date,时间间隔,年份
18393,525,4173466222,2015-08-06,3.0,2015-12-31,147,2015
20277,525,4194421248,2015-08-27,198.0,2015-12-31,126,2015
27563,525,4264661923,2015-11-24,12.0,2015-12-31,37,2015
37982,525,4349299441,2018-06-19,888.0,2018-12-31,195,2018


In [21]:
rfm_result.columns = ['年份', '会员ID', 'R', 'F', 'M']
rfm_result

,年份,会员ID,R,F,M
0,2015,267,197,2,105.0
1,2015,282,251,1,29.7
2,2015,283,340,1,5398.0
3,2015,343,300,1,118.0
4,2015,525,37,3,213.0
...,...,...,...,...,...
148586,2018,39538034299,272,1,49.0
148587,2018,39538034662,189,1,3558.0
148588,2018,39538035729,179,1,3699.0
148589,2018,39545237824,275,1,49.0


In [22]:
## 给RFM进行分组，本由业务方定分组规则；
## 当前这次业务，要给每一个维度打3、2、1；

In [23]:
# 查看统计数据，以便给业务方提供划分依据
rfm_result.describe()

,年份,会员ID,R,F,M
count,148591.000000,1.485910e+05,148591.000000,148591.000000,148591.000000
mean,2016.773075,2.811669e+10,165.524043,1.365002,1323.741329
std,1.129317,1.477660e+10,101.988472,2.626953,3753.906883
min,2015.000000,8.100000e+01,0.000000,1.000000,1.500000
25%,2016.000000,1.728262e+10,79.000000,1.000000,69.000000
50%,2017.000000,3.689151e+10,156.000000,1.000000,189.000000
75%,2018.000000,3.923337e+10,255.000000,1.000000,1199.000000
max,2018.000000,3.954614e+10,365.000000,130.000000,206251.800000


In [24]:
# R F M划分依据，当前是参考了数据的分布，R/M分别使用了1/4分位数，3/4分位数作为划分的阈值
# F由于行业的特点， 这里使用2， 5 作为划分的阈值
R_bins = [-1,79,255,365]  #[3, 2, 1]
F_bins = [0,2,5,130]  #[1,2,3]
M_bins = [1,69,1199,206252]  #[1,2,3]

In [25]:
# 分别对每RFM列进行打分，并生成新的列
rfm_result['r_score'] = pd.cut(rfm_result['R'], bins=R_bins, labels=[3, 2, 1])
rfm_result['f_score'] = pd.cut(rfm_result['F'], bins=F_bins, labels=[1, 2, 3])
rfm_result['m_score'] = pd.cut(rfm_result['M'], bins=M_bins, labels=[1, 2, 3])
rfm_result

,年份,会员ID,R,F,M,r_score,f_score,m_score
0,2015,267,197,2,105.0,2,1,2
1,2015,282,251,1,29.7,2,1,1
2,2015,283,340,1,5398.0,1,1,3
3,2015,343,300,1,118.0,1,1,2
4,2015,525,37,3,213.0,3,2,2
...,...,...,...,...,...,...,...,...
148586,2018,39538034299,272,1,49.0,1,1,1
148587,2018,39538034662,189,1,3558.0,2,1,3
148588,2018,39538035729,179,1,3699.0,2,1,3
148589,2018,39545237824,275,1,49.0,1,1,1


In [26]:
rfm_result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148591 entries, 0 to 148590
Data columns (total 8 columns):
 #   Column   Non-Null Count   Dtype   
---  ------   --------------   -----   
 0   年份       148591 non-null  int64   
 1   会员ID     148591 non-null  int64   
 2   R        148591 non-null  int64   
 3   F        148591 non-null  int64   
 4   M        148591 non-null  float64 
 5   r_score  148591 non-null  category
 6   f_score  148591 non-null  category
 7   m_score  148591 non-null  category
dtypes: category(3), float64(1), int64(4)
memory usage: 6.1 MB


In [27]:
# pd.cut默认返回一个category的类型，为了方便结果拼接，转换成字符串类型 
rfm_result['r_score'] = rfm_result['r_score'].astype(str)
rfm_result['f_score'] = rfm_result['f_score'].astype(str)
rfm_result['m_score'] = rfm_result['m_score'].astype(str)

In [28]:
rfm_result['rfm_label'] = rfm_result['r_score'] + rfm_result['f_score'] + rfm_result['m_score']
rfm_result

,年份,会员ID,R,F,M,r_score,f_score,m_score,rfm_label
0,2015,267,197,2,105.0,2,1,2,212
1,2015,282,251,1,29.7,2,1,1,211
2,2015,283,340,1,5398.0,1,1,3,113
3,2015,343,300,1,118.0,1,1,2,112
4,2015,525,37,3,213.0,3,2,2,322
...,...,...,...,...,...,...,...,...,...
148586,2018,39538034299,272,1,49.0,1,1,1,111
148587,2018,39538034662,189,1,3558.0,2,1,3,213
148588,2018,39538035729,179,1,3699.0,2,1,3,213
148589,2018,39545237824,275,1,49.0,1,1,1,111


In [29]:
rfm_result.groupby('rfm_label')['会员ID'].count()

rfm_label
111     9118
112    16846
113     7535
121        9
122     1015
123     1932
132        2
133      472
211    19023
212    36839
213    16370
221       24
222      374
223      370
232       13
233     1046
311     9274
312    18655
313     8341
321       34
322      410
323      366
331        1
332       37
333      485
Name: 会员ID, dtype: int64

## 5、结果保存和数据可视化

In [30]:
rfm_result.to_excel('./data/sale_rfm.xlsx', index=False)

In [31]:
rfm_result.columns

Index(['年份', '会员ID', 'R', 'F', 'M', 'r_score', 'f_score', 'm_score',
       'rfm_label'],
      dtype='object')

In [32]:
# 统计每一年， 每一个RFM组别中，分别有多少会员（会员数量）
display_data = rfm_result.groupby(['rfm_label', '年份'], as_index=False)['会员ID'].count()
display_data

,rfm_label,年份,会员ID
0,111,2015,2180
1,111,2016,1498
2,111,2017,3169
3,111,2018,2271
4,112,2015,3811
...,...,...,...
83,332,2018,24
84,333,2015,15
85,333,2016,28
86,333,2017,87


In [33]:
display_data.columns = ['rfm_group', 'year', 'number']
display_data

,rfm_group,year,number
0,111,2015,2180
1,111,2016,1498
2,111,2017,3169
3,111,2018,2271
4,112,2015,3811
...,...,...,...
83,332,2018,24
84,333,2015,15
85,333,2016,28
86,333,2017,87


In [34]:
# pyecharts 调用的是echarts 是百度开源的基于js的数据可视化库
# pyecharts 对pandas的数据结构支持的不好, 所以这里
# [d.tolist() for d in display_data.values] 把数据转换成 python的列表

In [35]:
display_data.head().values

array([['111', 2015, 2180],
       ['111', 2016, 1498],
       ['111', 2017, 3169],
       ['111', 2018, 2271],
       ['112', 2015, 3811]], dtype=object)

In [37]:
%pip install pyecharts
from pyecharts.charts import Bar3D  
from pyecharts import options as opts
# RGB   #313695  31 Red 36 Green 95 Blue  每一位都是16进制数
range_color = ['#313695', '#4575b4', '#74add1', '#abd9e9', '#e0f3f8', '#ffffbf',
               '#fee090', '#fdae61', '#f46d43', '#d73027', '#a50026']
range_max = int(display_data['number'].max())
c = (
    Bar3D()#设置了一个3D柱形图对象
    .add(
        "",#标题
        [d.tolist() for d in display_data.values],#数据
        xaxis3d_opts=opts.Axis3DOpts( type_="category",name='分组名称'),
        #x轴数据类型，category类别型 name x轴名称
        yaxis3d_opts=opts.Axis3DOpts( type_="category",name='年份'),
        #y轴数据类型，category类别型 name y轴名称
        zaxis3d_opts=opts.Axis3DOpts(type_="value",name='会员数量')
        ,#z轴数据类型，value 数值型 name z轴名称
    )
    .set_global_opts(#设置颜色，及不同取值对应的颜色
        visualmap_opts=opts.VisualMapOpts(max_=range_max,range_color=range_color),
        title_opts=opts.TitleOpts(title="RFM分组结果"),#设置标题
    )
)
c.render('rfm.html') #生成一个html页面 默认叫render.htm

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


'C:\\Users\\Admin\\Desktop\\john\\rfm.html'